In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from tqdm.auto import tqdm

import optuna
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import datasets, transforms
from torch.autograd import Variable
from torch.nn.modules.loss import _WeightedLoss

from library.Data_Preprocessing import Balance_Ratio
from library.Imbalance_Sampling import label_divide
from library.Aging_Score_Contour import score1
from library.AdaBoost import train_set, multiple_set, multiple_month, line_chart, AUC, PR_curve, multiple_curve, \
    best_threshold

os.chdir('C:/Users/user/Desktop/Darui_R08621110')  
os.getcwd()

'C:\\Users\\user\\Desktop\\Darui_R08621110'

## 

In [2]:
##### GPU ??? #####
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device.'.format(device))

Using cpu device.


### Dataloader

In [3]:
class RunhistSet(Dataset):
    
    def __init__(self, train_x, train_y):
        self.x = torch.tensor(train_x.values.astype(np.float32))
        self.y = torch.tensor(train_y.values.astype(np.float32)).long()
        
    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return [self.x[idx], self.y[idx]]

### Neural Network

In [4]:
class NeuralNetworkC(nn.Module):
    
    def __init__(self):
        super(NeuralNetworkC, self).__init__()
        self.stack = nn.Sequential(
            nn.Linear(114, 64),
            nn.ReLU(),
            nn.Dropout(0.25),
#             nn.Linear(64, 32),
#             nn.ReLU(),
#             nn.Dropout(0.25),
#             nn.Linear(64, 16),
#             nn.ReLU(),
#             nn.Dropout(0.25),
            nn.Linear(64, 2)
        )

    def forward(self, x):
        logits = self.stack(x)
        return logits
    
    
class NeuralNetworkR(nn.Module):
    
    def __init__(self):
        super(NeuralNetworkR, self).__init__()
        self.stack = nn.Sequential(
            nn.Linear(114, 64),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(64, 16),
            nn.ReLU(),
            nn.Dropout(0.25),
#             nn.Linear(32, 16),
#             nn.ReLU(),
#             nn.Dropout(0.25),
            nn.Linear(16, 1)
        )

    def forward(self, x):
        logits = self.stack(x)
        return logits

### label smoothing

In [5]:
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1, weight = None):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.weight = weight
        self.cls = classes
        self.dim = dim                                                                                                                                                                                                                                                                                                                                                                                                                                              

    def forward(self, pred, target):
        assert 0 <= self.smoothing < 1
        pred = pred.log_softmax(dim=self.dim)

        if self.weight is not None:
            pred = pred * self.weight.unsqueeze(0)   

        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

### training

In [19]:
def trainingC(network, trainloader, validloader, optimizer, criterion, epoch, filename, early_stop):
    
    network.train()
    best_model = network
    best_objective = 0
    stop_trigger = 0
    train_loss = []
    valid_loss = []
    
    for i in tqdm(range(epoch)):
        
        total_loss = 0
        TP = 0
        TN = 0
        FP = 0
        FN = 0  
        for x, y in trainloader:
            
            x = x.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            output = network(x)
            loss = criterion(output, y)         
            
            _, predicted = torch.max(output.data, 1)
            TP += torch.dot((predicted == y).to(torch.float32), (y == 1).to(torch.float32)).sum().item()
            TN += torch.dot((predicted == y).to(torch.float32), (y == 0).to(torch.float32)).sum().item()
            FN += torch.dot((predicted != y).to(torch.float32), (y == 1).to(torch.float32)).sum().item()
            FP += torch.dot((predicted != y).to(torch.float32), (y == 0).to(torch.float32)).sum().item()
            total_loss += loss.item()*len(y)
            loss.backward()
            optimizer.step()
        
        train_loss.append(total_loss)
        recall = TP / (TP + FN)
        aging = (TP + FP) / (TP + TN + FP + FN)
            
        print(f'Epoch {i+1}: Train Loss = {total_loss / (TP + TN + FP + FN)}, Recall = {recall}, Aging Rate = {aging}')
        
        if ((i+1) % 5 == 0):
            five_loss, valid_recall, _ = testingC(network, validloader, criterion)
            valid_loss.append(five_loss)
            
            if valid_recall > best_objective:
                best_objective = valid_recall
                best_model = network
                torch.save(best_model, f'{filename}_NeuralNetworkC_{epoch}.ckpt')
                print(f'Model in epoch {i+1} is saved.\n')
                stop_trigger = 0
            else:
                stop_trigger += 1
                print('')
                
            if stop_trigger == early_stop:
                print(f'Training Finished at epoch {i+1}.')
                return network, train_loss, valid_loss
            
    return network, train_loss, valid_loss


def PR_matrix(predict):
    
    Y_new = predict.sort_values(['predict', 'truth'], ascending = [False, True]).reset_index(drop = True)
    Y_new.loc[Y_new['truth'] != 1, 'truth'] = 0
    
    matrix = pd.DataFrame(Y_new.groupby('predict').sum()).rename(columns = {'truth': 'Bad_Count'})
    matrix = matrix.sort_index(ascending = False)
    matrix['All_Count'] = Y_new.groupby('predict').count()
    matrix['Class_Prob'] = matrix.index
    
    matrix['TP'] = matrix['Bad_Count'].cumsum()
    matrix['FP'] = matrix['All_Count'].cumsum() - matrix['TP']
    matrix['FN'] = matrix['TP'].values[-1] - matrix['TP']
    matrix['TN'] = matrix['FP'].values[-1] - matrix['FP']
    
    matrix['Precision'] = matrix['TP'] / (matrix['TP'] + matrix['FP'])
    matrix['Recall'] = matrix['TP'] / (matrix['TP'] + matrix['FN'])
    matrix['Aging Rate'] = (matrix['TP'] + matrix['FP']) / (matrix['TP'] + matrix['FP'] + matrix['FN'] + matrix['TN'])
    matrix['Efficiency'] = matrix['Recall'] / matrix['Aging Rate']
    matrix['Score'] = score1(matrix['Recall'], matrix['Aging Rate'])            
    matrix = matrix.drop(columns = ['Bad_Count', 'All_Count']).reset_index(drop = True)
    
    return matrix


def trainingR(network, trainloader, validloader, optimizer, criterion, epoch, filename, early_stop):
    
    network.train()
    best_model = network
    best_objective = 1
    stop_trigger = 0
    train_loss = []
    valid_loss = []
    
    for i in tqdm(range(epoch)):
        
        total_loss = 0
        predict_vector = torch.tensor([0])
        y_vector = torch.tensor([0])
        for x, y in trainloader:
            
            x = x.to(device)
            y = y.type(torch.FloatTensor).to(device)
            y = y.unsqueeze(1)
            optimizer.zero_grad()
            output = network(x)
            loss = criterion(output, y)
            total_loss += loss.item()*len(y)
            loss.backward()
            optimizer.step()    
            predict_vector = torch.cat((predict_vector, output.data[:,0]), axis = 0)
            y_vector = torch.cat((y_vector, y[:,0]), axis = 0)       
        result_df = pd.DataFrame(dict(predict = predict_vector, truth = y_vector))
        pr_matrix = PR_matrix(result_df.iloc[1:, :])
        best_data, best_thres = best_threshold(pr_matrix, target = 'Recall', threshold = 0.7)
        auc = AUC(pr_matrix['Recall'], pr_matrix['Aging Rate'])
        train_loss.append(total_loss)
        
        recall = best_data["Recall"].values
        aging = best_data["Aging Rate"].values
        print(f'Epoch {i+1}: Train Loss = {total_loss}, AUC = {auc}, Recall(0.7) = {recall}, Aging Rate = {aging}')
        
        if ((i+1) % 5 == 0):
            five_loss, valid_auc, _ = testingR(network, validloader, criterion)
            valid_loss.append(five_loss)
            
            if valid_auc < best_objective:
                best_objective = valid_auc
                best_model = network
                torch.save(best_model, f'{filename}_NeuralNetworkR_{epoch}.ckpt')
                print(f'Model in epoch {i+1} is saved.\n')
                stop_trigger = 0
            else:
                stop_trigger += 1
                print('')
                
            if stop_trigger == early_stop:
                print(f'Training Finished at epoch {i+1}.')
                return network, train_loss, valid_loss
      
    return network, train_loss, valid_loss

### testing

In [18]:
def testingC(network, dataloader, criterion):
    
    network.eval()
    total_loss = 0
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    
    for x, y in dataloader:
        
        x = x.to(device)
        y = y.to(device)
        output = network(x)
        loss = criterion(output, y)
        
        _, predicted = torch.max(output.data, 1)
        TP += torch.dot((predicted == y).to(torch.float32), (y == 1).to(torch.float32)).sum().item()
        TN += torch.dot((predicted == y).to(torch.float32), (y == 0).to(torch.float32)).sum().item()
        FN += torch.dot((predicted != y).to(torch.float32), (y == 1).to(torch.float32)).sum().item()
        FP += torch.dot((predicted != y).to(torch.float32), (y == 0).to(torch.float32)).sum().item()
        total_loss += loss.item()*len(y)
        
    recall = TP / (TP + FN)
    aging = (TP + FP) / (TP + TN + FP + FN)
    if (TP + FP) != 0:
        precision = TP / (TP + FP)
    else:
        precision = 0
    if aging != 0:
        efficiency = recall / aging
        score = score1(recall, aging)
    else:
        efficiency = 0
        score = 0
        
    print(f'Test Loss = {total_loss / (TP + TN + FP + FN)}, Recall = {recall}, Aging Rate = {aging}, Efficiency = {recall / (aging + 1e-8)}')
    
    valid_objective = recall - 0.5*aging
    table = pd.Series({'TP': TP, 'FP': FP, 'FN': FN, 'TN': TN, 'Precision': precision, 'Recall': recall, 
                       'Aging Rate': aging, 'Efficiency': efficiency, 'Score': score})
    table = pd.DataFrame(table).T
    
    return total_loss, valid_objective, table


def testingR(network, dataloader, criterion):
    
    network.eval()   
    total_loss = 0
    predict_vector = torch.tensor([0])
    y_vector = torch.tensor([0])
    for x, y in dataloader:
        
        x = x.to(device)
        y = y.to(device)
        y = y.unsqueeze(1)
        output = network(x)
        loss = criterion(output, y)
        total_loss += loss.item()*len(y)
        
        predict_vector = torch.cat((predict_vector, output.data[:,0]), axis = 0)
        y_vector = torch.cat((y_vector, y[:,0]), axis = 0)
    result_df = pd.DataFrame(dict(predict = predict_vector, truth = y_vector))
    pr_matrix = PR_matrix(result_df.iloc[1:, :])
    best_data, best_thres = best_threshold(pr_matrix, target = 'Recall', threshold = 0.7)
    auc = AUC(pr_matrix['Recall'], pr_matrix['Aging Rate'])
    recall = best_data['Recall'].values[0]
    aging = best_data['Aging Rate'].values[0]
    precision = best_data['Precision'].values[0]
    efficiency = best_data['Efficiency'].values[0]
    score = best_data['Score'].values[0]
    TP = best_data['TP'].values[0]
    FP = best_data['FP'].values[0]
    TN = best_data['TN'].values[0]
    FN = best_data['FN'].values[0]
        
    print(f'Test Loss = {total_loss}, Recall = {recall}, Aging Rate = {aging}, Efficiency = {efficiency}')
    
    valid_objective = auc
    table = pd.Series({'TP': TP, 'FP': FP, 'FN': FN, 'TN': TN, 'Precision': precision, 'Recall': recall,
                       'Aging Rate': aging,'Efficiency': efficiency, 'Score': score})
    table = pd.DataFrame(table).T
    
    return total_loss, valid_objective, table

### Run all datasets

In [8]:
def runall_nn(train_x, train_y, test_x, test_y, n_epoch, batch_size, model, optimizer, criterion, filename,
              train_ratio, early_stop, mode):
    
    set_name = list(train_x.keys())
    result_table = pd.DataFrame()
    train_dict = {}
    valid_dict = {}
    for num, i in enumerate(tqdm(set_name)):
        print(f'\nStarting training Dataset {num}:')
        
        # data preparation
        train_ratio = train_ratio
        train_data = RunhistSet(train_x[i], train_y[i])
        test_data = RunhistSet(test_x, test_y)
        train_size = int(len(train_data)*train_ratio)
        valid_size = len(train_data) - train_size
        train_data, valid_data = random_split(train_data, [train_size, valid_size])
        
        train_loader = DataLoader(train_data, batch_size = batch_size, shuffle = True)
        valid_loader = DataLoader(valid_data, batch_size = batch_size, shuffle = False)
        test_loader = DataLoader(test_data, batch_size = batch_size, shuffle = False)
        
        # training
        if mode == 'C':
            done_model, train_loss, valid_loss = trainingC(network = model, 
                                                           trainloader = train_loader, 
                                                           validloader = valid_loader, 
                                                           optimizer = optimizer, 
                                                           criterion = criterion, 
                                                           epoch = n_epoch, 
                                                           filename = filename, 
                                                           early_stop = early_stop)
        elif mode == 'R':
            done_model, train_loss, valid_loss = trainingR(network = model, 
                                                           trainloader = train_loader, 
                                                           validloader = valid_loader, 
                                                           optimizer = optimizer, 
                                                           criterion = criterion, 
                                                           epoch = n_epoch, 
                                                           filename = filename, 
                                                           early_stop = early_stop)
        train_dict[i] = train_loss
        valid_dict[i] = valid_loss
        
        # testing
        if mode == 'C':
            _, _, table = testingC(done_model, test_loader, criterion)
        elif mode == 'R':
            _, _, table = testingR(done_model, test_loader, criterion)
        result_table = pd.concat([result_table, table], axis = 0).rename({0: f'dataset {num}'})
    loss_dict = dict(train = train_dict, valid = valid_dict)
        
    return result_table, loss_dict


def loss_plot(train_loss, valid_loss, num_row, num_col):
    
    fig , axes = plt.subplots(num_row, num_col, sharex = False, sharey = False, figsize = (num_row*8 + 1, num_col*6))
    plt.suptitle('Training & Validation Loss Curve', y = 0.94, fontsize = 30)
    for row in range(num_row):
        for col in range(num_col):
            
            index = num_col*row + col
            if index < len(train_loss):
                
                train = train_loss[f'set{index}']
                valid = valid_loss[f'set{index}']
                axes[row, col].plot(range(len(train)), train, 'b-', linewidth = 5, label = 'train')
                axes[row, col].plot(range(4, len(train)+1, 5), valid, 'r-', linewidth = 5, label = 'valid')
                axes[row, col].set_xlabel('Epoch')
                axes[row, col].set_ylabel('Total Loss')
                axes[row, col].set_title(f'dataset {index}')
                axes[row, col].legend(loc = 'upper right', fancybox = True, prop = dict(size = 20))

## 

### loading training & testing data

In [9]:
### training data ### 
training_month = [2, 3, 4]

data_dict, trainset_x, trainset_y = multiple_month(training_month, num_set = 10, filename = 'dataset')

print('\nCombined training data:\n')
run_train = multiple_set(num_set = 10)
run_train_x, run_train_y = train_set(run_train, num_set = 10)

### testing data ###
run_test = pd.read_csv('test_runhist.csv').iloc[:, 2:]
run_test_x, run_test_y = label_divide(run_test, None, 'GB', train_only = True)
print('\n', 'Dimension of testing data:', run_test.shape)


Month 2:

Dimension of dataset 0 : (38022, 96)  balance ratio: 863.0
Dimension of dataset 1 : (1128, 96)  balance ratio: 1.0
Dimension of dataset 2 : (962, 96)  balance ratio: 1.0
Dimension of dataset 3 : (1226, 96)  balance ratio: 1.0
Dimension of dataset 4 : (1138, 96)  balance ratio: 1.0
Dimension of dataset 5 : (883, 96)  balance ratio: 1.0
Dimension of dataset 6 : (921, 96)  balance ratio: 1.0
Dimension of dataset 7 : (880, 96)  balance ratio: 1.0
Dimension of dataset 8 : (880, 96)  balance ratio: 1.0
Dimension of dataset 9 : (484, 96)  balance ratio: 10.0

 10 datasets are loaded.

Labels of  10 datasets are divided.

Month 3:

Dimension of dataset 0 : (60607, 107)  balance ratio: 550.0
Dimension of dataset 1 : (1800, 107)  balance ratio: 1.0
Dimension of dataset 2 : (2948, 107)  balance ratio: 1.0
Dimension of dataset 3 : (2030, 107)  balance ratio: 1.0
Dimension of dataset 4 : (1814, 107)  balance ratio: 1.0
Dimension of dataset 5 : (2203, 107)  balance ratio: 1.0
Dimension of

### For one dataset

In [10]:
##### data preparation #####
train_data = RunhistSet(run_train_x['set7'], run_train_y['set7'])
test_data = RunhistSet(run_test_x, run_test_y)
train_ratio = 0.75
train_size = int(len(train_data)*train_ratio)
valid_size = len(train_data) - train_size
train_data, valid_data = random_split(train_data, [train_size, valid_size])

train_loader = DataLoader(train_data, batch_size = 64, shuffle = True)
valid_loader = DataLoader(valid_data, batch_size = 64, shuffle = False)
test_loader = DataLoader(test_data, batch_size = 64, shuffle = False)

#### classifier

In [ ]:
##### model preparation #####
# hyperparameter: learning rate, weight decay, weight
modelC = NeuralNetworkC().to(device)
optimizerC = torch.optim.Adam(modelC.parameters(), lr = 0.001, weight_decay = 0.01)
criterionC = nn.CrossEntropyLoss(weight = torch.tensor([0.5, 0.5])).to(device)

criterionC = LabelSmoothingLoss(classes = 2, smoothing = 0.2)

##### training #####
done_modelC, train_lossC, valid_lossC = trainingC(network = modelC, 
                                                  trainloader = train_loader, 
                                                  validloader = valid_loader, 
                                                  optimizer = optimizerC, 
                                                  criterion = criterionC, 
                                                  epoch = 150, 
                                                  filename = 'tamama',
                                                  early_stop = 10)

##### testing #####
_, _, result_tableC = testingC(done_modelC, test_loader, criterionC)

#### regressor

In [20]:
##### model preparation #####
# hyperparameter: learning rate, weight decay, weight
modelR = NeuralNetworkR().to(device)
optimizerR = torch.optim.Adam(modelR.parameters(), lr = 0.001, weight_decay = 0.001)
criterionR = nn.MSELoss().to(device)

##### training #####
done_modelR, train_lossR, valid_lossR = trainingR(network = modelR, 
                                                  trainloader = train_loader, 
                                                  validloader = valid_loader, 
                                                  optimizer = optimizerR, 
                                                  criterion = criterionR, 
                                                  epoch = 150, 
                                                  filename = 'tamama',
                                                  early_stop = 10)

##### testing #####
_, _, result_tableR = testingR(done_modelR, test_loader, criterionR)

  0%|          | 0/150 [00:00<?, ?it/s]

Best Threshold: 0.43432676792144775 

Recall: [0.70041752] ,   Precision: [0.63361662] ,   Aging Rate: [0.54517375]
Epoch 1: Train Loss = 846.9388972967863, AUC = 0.3952542841727463, Recall(0.7) = [0.70041752], Aging Rate = [0.54517375]
Best Threshold: 0.5488395690917969 

Recall: [0.70041752] ,   Precision: [0.94908062] ,   Aging Rate: [0.36396396]
Epoch 2: Train Loss = 518.1432443782687, AUC = 0.27935565393543216, Recall(0.7) = [0.70041752], Aging Rate = [0.36396396]
Best Threshold: 0.6151427626609802 

Recall: [0.70041752] ,   Precision: [0.98676471] ,   Aging Rate: [0.35006435]
Epoch 3: Train Loss = 382.7067674770951, AUC = 0.26039226766551476, Recall(0.7) = [0.70041752], Aging Rate = [0.35006435]
Best Threshold: 0.6658647656440735 

Recall: [0.70041752] ,   Precision: [0.99628805] ,   Aging Rate: [0.34671815]
Epoch 4: Train Loss = 303.71694777160883, AUC = 0.2535262898694735, Recall(0.7) = [0.70041752], Aging Rate = [0.34671815]
Best Threshold: 0.6969634890556335 

Recall: [0.7004

Best Threshold: 0.9722369313240051 

Recall: [0.70041752] ,   Precision: [1.] ,   Aging Rate: [0.34543115]
Epoch 32: Train Loss = 9.031352465739474, AUC = 0.24658951385088584, Recall(0.7) = [0.70041752], Aging Rate = [0.34543115]
Best Threshold: 0.9751859903335571 

Recall: [0.70041752] ,   Precision: [1.] ,   Aging Rate: [0.34543115]
Epoch 33: Train Loss = 8.162652850616723, AUC = 0.24658951382979027, Recall(0.7) = [0.70041752], Aging Rate = [0.34543115]
Best Threshold: 0.9743627309799194 

Recall: [0.70041752] ,   Precision: [1.] ,   Aging Rate: [0.34543115]
Epoch 34: Train Loss = 8.170216631609946, AUC = 0.24658951382211916, Recall(0.7) = [0.70041752], Aging Rate = [0.34543115]
Best Threshold: 0.9755672216415405 

Recall: [0.70041752] ,   Precision: [1.] ,   Aging Rate: [0.34543115]
Epoch 35: Train Loss = 8.700050393119454, AUC = 0.24658951382979027, Recall(0.7) = [0.70041752], Aging Rate = [0.34543115]
Best Threshold: 0.9651535153388977 

Recall: [0.70178042] ,   Precision: [1.] , 

Best Threshold: 0.9831069707870483 

Recall: [0.70041752] ,   Precision: [1.] ,   Aging Rate: [0.34543115]
Epoch 63: Train Loss = 5.964000734151341, AUC = 0.24658951382979027, Recall(0.7) = [0.70041752], Aging Rate = [0.34543115]
Best Threshold: 0.982894778251648 

Recall: [0.70041752] ,   Precision: [1.] ,   Aging Rate: [0.34543115]
Epoch 64: Train Loss = 6.6053667610976845, AUC = 0.24658951382979027, Recall(0.7) = [0.70041752], Aging Rate = [0.34543115]
Best Threshold: 0.9815890789031982 

Recall: [0.70041752] ,   Precision: [1.] ,   Aging Rate: [0.34543115]
Epoch 65: Train Loss = 5.97409304860048, AUC = 0.24658951382787245, Recall(0.7) = [0.70041752], Aging Rate = [0.34543115]
Best Threshold: 0.9652794599533081 

Recall: [0.70029674] ,   Precision: [1.] ,   Aging Rate: [0.36447876]
Test Loss = 14.8444560226053, Recall = 0.7002967359050445, Aging Rate = 0.3644787644787645, Efficiency = 1.9213649851632046

Best Threshold: 0.9824767112731934 

Recall: [0.70041752] ,   Precision: [1.] ,

### For multiple datasets

#### classifier

In [ ]:
runall_modelC = NeuralNetworkC().to(device)
runall_optimizerC = torch.optim.Adam(runall_modelC.parameters(), lr = 0.001, weight_decay = 0.01)
runall_criterionC = nn.CrossEntropyLoss(weight = torch.tensor([0.5, 0.5])).to(device)

table_setC, loss_dictC = runall_nn(train_x = run_train_x, 
                                   train_y = run_train_y, 
                                   test_x = run_test_x, 
                                   test_y = run_test_y, 
                                   n_epoch = 150, 
                                   batch_size = 64,
                                   model = runall_modelC,
                                   optimizer = runall_optimizerC, 
                                   criterion = runall_criterionC, 
                                   filename = 'runhist_array_4criteria_NeuralNetworkC', 
                                   train_ratio = 0.75, 
                                   early_stop = 10,
                                   mode = 'C')

In [ ]:
loss_plot(loss_dictC['train'], loss_dictC['valid'], num_row = 4, num_col = 3)
table_setC

#### regressor

In [ ]:
runall_modelR = NeuralNetworkR().to(device)
runall_optimizerR = torch.optim.Adam(runall_modelR.parameters(), lr = 0.001, weight_decay = 0.001)
runall_criterionR = nn.MSELoss().to(device)

table_setR, loss_dictR = runall_nn(train_x = run_train_x, 
                                   train_y = run_train_y, 
                                   test_x = run_test_x, 
                                   test_y = run_test_y, 
                                   n_epoch = 150, 
                                   batch_size = 64,
                                   model = runall_modelR,
                                   optimizer = runall_optimizerR, 
                                   criterion = runall_criterionR, 
                                   filename = 'runhist_array_4criteria_NeuralNetworkR', 
                                   train_ratio = 0.75, 
                                   early_stop = 10,
                                   mode = 'R')

In [ ]:
loss_plot(loss_dictR['train'], loss_dictR['valid'], num_row = 4, num_col = 3)
table_setR

### export

In [ ]:
savedate = '20211102'
TPE_multi = True

table_setC['sampler'] = 'multivariate-TPE' if TPE_multi else 'univariate-TPE'
table_setR['sampler'] = 'multivariate-TPE' if TPE_multi else 'univariate-TPE'
table_setC['model'] = 'NeuralNetwork'
table_setR['model'] = 'NeuralNetwork'
with pd.ExcelWriter(f'{savedate}_Classifier.xlsx', mode = 'a') as writer:
    table_setC.to_excel(writer, sheet_name = 'NeuralNetwork')
with pd.ExcelWriter(f'{savedate}_Regressor.xlsx', mode = 'a') as writer:
    table_setR.to_excel(writer, sheet_name = 'NeuralNetwork')